<a href="https://colab.research.google.com/github/jlschwartz3/bio108tutorial/blob/main/Data_Science_Coding_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Rain, rain, go away? Or rain, rain, come?
##Precipitation, climate variability, and global agriculture


---



Across the world, scientists are working to predict the effects of climate change on spatial and temporal levels. Global climate models, or GCMs, attempt to mathematically represent the complex processes of the Earth's climate system, including land surface, oceans, and the atmosphere. In the face of accelerating climate change, these models can be used to predict future climate data.

Globally, cycles of rainfall dictate when to sow and harvest crops. Expected precipitation values for given months are essential to planning tilling, purchasing seeds, and determining if certain crops can continue to thrive in certain regions.

Using projected climate data, we will examine how precipitation seasonality varies across regions and how that could impact agricultural patterns in the next 2 decades. We'll also take a look at the relationship between precipitation and climate variability.

The data come from WorldClim's future climate data projections for 2021-2040. It comes from the EC-Earth3-Veg GCM, a model that focuses on changes in vegetation types and distribution in response to climate change, and uses the Shared Socio-economic Pathway (SSP) 126, which refers to a low-emissions projection of future radiative forcing. The data is at the 10 minute spatial resolution, the most coarse of WorldClim's data.

For today's tutorial, we will start with working with raster data, where pixels are organized into larger grids to display geographic information.  

In [ ]:
! pip install rasterio fiona #install rasterio library, which is not pre-loaded into Colab

! pip install -U leafmap #install leafmap, fiona, and rasterio packages
! pip install fiona
! pip install folium matplotlib mapclassify
! pip install localtileserver
! pip install rasterio
! pip install geopandas rioxarray rasterio

#install gdown to read in large files that were incompatible with GitHub
!pip install -q gdown

In [41]:
import rasterio #install all necessary packages for working with raster data
import rasterio.plot
import matplotlib.pyplot as plt

Accessing files from GitHub is best practice, but GitHub has some size limitations that can be tricky with large, spatial data. As a workaround, these files are linked to a publicly accessible Google Drive folder, and can be directly downloaded.

In [ ]:
#import gdown
import gdown

#download the first .tif file on global precip
gdown.download('https://drive.google.com/uc?id=1KRO6U7HHbdw_4KDmbmbLHn6tdrSZsZOR', 'wc2.1_10m_prec_EC-Earth3-Veg_ssp126_2021-2040.tif', quiet=False)

#download the second .tif file on maize crop yields
gdown.download('https://drive.google.com/uc?id=1Y4kW_Ig2-Op-COJVw0nMLqEZsqeqx8k2', 'explanatorycat_maize.tif', quiet=False)

In [ ]:
raster_path = (
    "wc2.1_10m_prec_EC-Earth3-Veg_ssp126_2021-2040.tif"
)
#read in the raster data from .tif file, which was uploaded into Drive for ease of download
src = rasterio.open(raster_path)
print(src)

Let's take a look at the metadata! There is an important number here, given that we're looking at years (and months). Any guesses?

```
src.meta #examine metadata. Important: shows coordinate reference system.
```

Now, let's plot the data and include a colormap to better visualize it.

```
fig, ax = plt.subplots(figsize=(8, 8))
rasterio.plot.show(src, cmap="viridis_r", ax=ax)
plt.show()
```

Next, we'll plot the data separated out by band (month).
```
band_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"] #name all of the 12 bands present in the
#dataset, which correspond to the 12 months of the year

fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(10, 18)) #create 6 rows and 2 columns for clear visual output
axes = axes.flatten()  #flatten the 2D array of axes to 1D for easy iteration

for band in range(1, src.count + 1): #include the +1 so that the final band is still included; all bands will be plotted in separate plots
    data = src.read(band)
    ax = axes[band - 1]
    im = ax.imshow(data, cmap="viridis_r", vmin=0, vmax=1000) #use viridis_r colormap to show contrast between yellow (low precip) and blue/green (higher precip)
    ax.set_title(f"Band {band_names[band - 1]}")
    fig.colorbar(im, ax=ax, label="Precipitation (mm)", shrink=0.25) #add in colorbar scale to help interpret precipitation values

plt.tight_layout()
plt.show()
```

### Geographic clipping

For the next analysis, let's zoom in on a specific country to take a closer look at its unique precipitation patterns: Brazil.

As of 2022, Brazil was the world's second-largest exporter of agricultural products and provides food security to much of Latin America. Its soybeans, corn, beef, and coffee are are also staples for China, which sources nearly a quarter of its imports from Brazil. Crops coming from Brazil feed approximately 10% of the the world.

From a climate perspective, Brazil plays a crucial role in the regulation of global patterns through the influence of the Amazon rainforest, much of which lies within Brazil's boundaries. Evapotranspiration from the rainforest results in increased water vapor in the atmosphere, which condenses and falls as precipitation near and far, depending on wind patterns. The Amazon rainforest is responsible for around 50% of all rainfall in the region, supplying moisture to farmers across the basin.

How much does Brazil's rainfall fluctuate throughout the year? To optimize agriculture, it's important for farmers to have a strong understanding of precipitation patterns for each month.

Here, we're going to read in a GeoJSON file, which is a type of vector file, or spatial data shown through lines, points, and polygons. In this case, it's a polygon representing the boundaries of Brazil that's publicly available.

```
import fiona
import rasterio.mask
import geopandas as gpd

geojson_path = "https://raw.githubusercontent.com/jlschwartz3/bio108tutorial/refs/heads/main/Downloads/Brazil%20Country%20Boundary.geojson/brazil_Brazil_Country_Boundary.geojson"
bounds = gpd.read_file(geojson_path) #import geoJSON vector file showing Brazil's national boundaries

fig, ax = plt.subplots()
rasterio.plot.show(src, ax=ax)
bounds.plot(ax=ax, edgecolor="red", facecolor="none") #plot the original plot, with the Brazil boundary vector overlaid
```



Here, we'll clip the precipitation data so that only Brazil is shown.

```
import matplotlib.pyplot as plt
import numpy as np

with fiona.open(geojson_path, "r") as f:
    shapes = [feature["geometry"] for feature in f]

out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True) #clips raster file to just show the desired feature (Brazil)

global_vmin = np.nanmin(out_image)  #minimum value across all bands
global_vmax = np.nanmax(out_image)  #maximum value across all bands

num_bands = out_image.shape[0]

fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(10, 18))
axes = axes.flatten()

for i in range(num_bands):
    band_data = out_image[i]

    ax = axes[i]
    im = ax.imshow(band_data, cmap="viridis_r", vmin=global_vmin, vmax=global_vmax)
    ax.set_title(f"Band {i + 1}") #bands correspond to months of the year
    cbar = plt.colorbar(im, ax=ax, shrink=0.6)
    cbar.set_label("Precipitation (mm)", fontsize=10)


plt.tight_layout()
plt.show()
```

The clipped raster files show that precipitation is generally heaviest in the December-April months, but that it varies by region. Certain parts in the northwestern area, which is densely forested, are rainiest May-July, whereas some heavily agricultural zones in the south-central region have a shorter "wet season," with heaviest precipitation concentrated in just December and January.

What other patterns do you see?

# Climatic variation and crop yield



So how does this connect to conservation of agriculture and climate variability?

In a paper published in Nature Communications in 2015, [Deepak Ray et al.](https://doi.org/10.1038/ncomms6989) examined the influence of climatic variation on fluctuation of yields for major global crops.

They found that climate variability accounts for approximately one-third (32–39%) of the observed annual yield variability for maize, rice, wheat, and soybean globally. This is huge!

Furthermore, the impact of climate factors varies by region. In some places, temperature is more influential, whereas precipitation is more important in others.

For our analysis, we'll focus on just precipitation.



Citation:
Ray, D., Gerber, J., MacDonald, G. et al. Climate variation explains a third of global crop yield variability. Nat Commun 6, 5989 (2015). https://doi.org/10.1038/ncomms6989




### **Our objectives**


1. Visualize the global patterns of crop yield variability based on the time series data using Leafmap.
2. Narrow this analysis to Brazil, a key producer of maize, to compare with earlier precipitation analysis.
3. Analyze whether total crop yield variability explained due to climate variability is higher in wetter or drier regions within Brazil.

To execute this analysis, we will use the same dataset that Ray et al. 2015 used to produce their data. It's available in EarthStat's collection of geographic datasets on land use and agriculture.

We will use the Leafmap package to create an interactive visualization.

Leafmaps show changes in data using color scales. Because Ray et al.'s data is categorical, the colors can't be represented as a continuous scale, like in the previous visualizations. Instead, they identified 8 different categories, and all of the data will fall into one of those 8 "buckets" and be represented by a very defined color.

Their categories were: No Data/Ocean, No Effect, 0–15%, >15–30%, >30–45%, >45–60%, >60–75%, >75%, with the percentages representing the percent of crop yield variability that was explained by variation in climate.

In [49]:
import leafmap #import packages
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# define category values and labels (including 0 as no data/ocean)
categories = [0, 1, 2, 3, 4, 5, 6, 7]
labels = [
    "No Data / Ocean", "No Effect", "0–15%", ">15–30%", ">30–45%", ">45–60%", ">60–75%", ">75%"
]

# define colors and set gray as color for no data/ocean areas
spectral_cmap = plt.get_cmap('Spectral')
color_values = [mcolors.to_rgba('lightgray')] + [spectral_cmap(i / 6) for i in range(7)]
cmap = mcolors.ListedColormap(color_values)
vmin = 0
vmax = 7

```
#path to the .tif file on total crop yield variability explained due to climate variability for maize
tif = 'explanatorycat_maize.tif'
#create the map
m = leafmap.Map(center=(33, 12), zoom=3, height="590px")

#add in the colorbar
m.add_raster(
    tif,
    layer_name="Maize Raster", #add the raster for format with only 1 band and no band descriptions
    cmap=cmap,
    vmin=vmin,
    vmax=vmax,
    nodata=0,  # optional if 0 is no data
    opacity=1,
)

#add in text to legend
legend_dict = dict(zip(labels, [mcolors.to_hex(c) for c in color_values]))
m.add_legend(title="Maize Variability Explained by Climate Variability", legend_dict=legend_dict)

m
```

The resulting Leafmap shows that in darker blue and green regions, climate variability is more strongly correlated with variability in crop yields. Based on a variety of variable feeding into their model, temperature and precipitation changes were more likely to explain changing crop yields.

Red and yellow regions indicate that climate variability is less strongly correlated with crop yield variability. In these areas, changes in precipitation and temperature were less responsible for changes in output.

Next, we'll perform the same raster clipping operation as earlier using a vector file on Brazil's boundaries to narrow the scope of this analysis for maize production, analyzing whether crop variability as explained by climate variability differs in wetter and drier areas.

###Maize

In [51]:
import geopandas as gpd
import rioxarray as rxr
import xarray as xr

# load in the Brazil boundary vector file
brazil = gpd.read_file("https://raw.githubusercontent.com/jlschwartz3/bio108tutorial/refs/heads/main/Downloads/Brazil%20Country%20Boundary.geojson/brazil_Brazil_Country_Boundary.geojson")

# load the raster and make sure it has spatial info
raster = rxr.open_rasterio(
    "explanatorycat_maize.tif",
    masked=True
)

```
#check that the CRS matches (if not, reproject Brazil boundary)
if raster.rio.crs != brazil.crs:
    brazil = brazil.to_crs(raster.rio.crs)

#clip the raster to Brazil
raster_clipped = raster.rio.clip(brazil.geometry, brazil.crs, drop=True)

#save the clipped raster
raster_clipped.rio.to_raster("maize_clipped_brazil.tif")
```

To plot the Leafmap using the Brazil vector boundaries, we can run the earlier leafmap code, but with the newly clipped data.

```
m = leafmap.Map(center=(-10, -52), zoom=4)  #centered around Brazil

m.add_raster(
    "maize_clipped_brazil.tif",
    layer_name="Brazil Maize",
    cmap=cmap,
    vmin=vmin,
    vmax=vmax,
    nodata=0,
)

m.add_legend(title="Maize Variability Explained by Climate Variability", legend_dict=legend_dict)
m
```

With our Leafmap now clipped to just show Brazil, we can use this data to "overlay" it with the original precipitation data, and examine the relationships between these 2 variables. Basically, it makes a copy of this smaller subset of data, saves this as a new .tif file, and loads both the precip raster and maize raster.

In [55]:
import rasterio

out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)

#use the original data and update it for the new clipped raster
out_meta = src.meta.copy()

out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

#save the clipped precip raster
with rasterio.open("precip_brazil.tif", "w", **out_meta) as dest:
    dest.write(out_image)

import rioxarray as rxr

precip = rxr.open_rasterio("precip_brazil.tif", masked=True).squeeze()

#load both rasters
yield_var = rxr.open_rasterio("maize_clipped_brazil.tif", masked=True).squeeze()
precip = rxr.open_rasterio("precip_brazil.tif", masked=True).squeeze()

#reproject precipitation to match yield raster
precip = precip.rio.reproject_match(yield_var)

Next, we'll calculate the precipitation values from within the clipped raster. The valid_mask object finds every place where both rasters have real values, and the .values pulls the real values out of the pixel form. Now, the data is in the form of an array, which is easier to work with for numerical calculations.

```
import numpy as np
import xarray as xr

#mask where there's no data in either
valid_mask = (~yield_var.isnull()) & (~precip.isnull())

#extract all valid values
yield_vals = yield_var.where(valid_mask).values.flatten()
precip_vals = precip.where(valid_mask).values.flatten()
```

Now, we'll find the average precipitation for each of the 7 categories identified by Ray et al. representing the influence of climate variability. We're excluding anything with missing data and then grouping the data by yield variability category, and then calculating the average for each.

```
import pandas as pd

df = pd.DataFrame({
    "yield_var_cat": yield_vals.astype(int),
    "precip": precip_vals
})

#keep only yield_var_cat values between 1 and 7, excluding 0/No Data areas because of difficulty wrangling
df = df[df["yield_var_cat"].between(1, 7)]

#remove NaNs, in case precip still has them
df = df.dropna()

#group by yield variability class
avg_precip_by_class = df.groupby("yield_var_cat")["precip"].mean()
print(avg_precip_by_class)
```

Finally, we'll make a simple bar graph to represent this data. Why is a bar graph a good choice here?


```
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 5))

avg_precip_by_class.plot.bar(
    ax=ax,
    title="Avg Precip by Maize Yield Variability Explained by Climate Variability"
)

ax.set_ylabel("Average Monthly Precipitation (mm)")  
ax.set_xlabel("Variability Class")      
plt.show()
```

#### **Questions**

1. What do we observe in this final graph? Does there seem to be a pattern?
2. In areas where climate (and precip) variability has more of an impact, how can farmers protect themselves against climate change? How can governments help?
3. Taking a step back, if farmers in certain regions are experiencing a lot of variability in crop yields, how might they adjust their decisions on sowing and harvesting? Could it drive them to abandon farming altogether?
